In [2]:
import pandas as pd
import nltk
import re
import string
from collections import defaultdict
import numpy as np
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os
import threading
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
import time
from timeit import default_timer as timer

basic_stopwords = stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()
snowball_stemmer = SnowballStemmer('english')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

basic_stopwords = stopwords.words('english')

df = pd.read_csv("dataset2.csv",encoding='latin')
df = df.drop(df.columns[[1,2,3,4]],axis=1)
df = df.rename(columns={df.columns[0]: 'target', df.columns[1]: 'tweet'})

closed_categories = ['CC', 'CD', 'DT', 'EX', 'IN', 'LS', 'MD', 'PDT', 'POS', 'PRP', 'PRP$', 'RP', 'TO', 'UH', 'WDT',
                     'WP', 'WP$', 'WRB']

vocabulary = []

def custom_lemmatizer(word, pos_tag): #takes as argument the word and the pos tag

    flag = 0
    for tag in closed_categories:
        if pos_tag == tag: #if pos tag is a close category tag, the functions returns flag with value 1
            flag = 1
            return word
    if flag == 0: #if not , the word gets lemmatized according to its part of speech
        if pos_tag.startswith("N"):
            word = wordnet_lemmatizer.lemmatize(word, wordnet.NOUN)
#             vocabulary.append(word)
            return (word)
        elif pos_tag.startswith('V'):
            word = wordnet_lemmatizer.lemmatize(word, wordnet.VERB)
#             vocabulary.append(word)
            return (word)
        elif pos_tag.startswith('J'):
            word = wordnet_lemmatizer.lemmatize(word, wordnet.ADJ)
            vocabulary.append(word)
            return (word)
        elif pos_tag.startswith('R'):
            word = wordnet_lemmatizer.lemmatize(word, wordnet.ADV)
#             vocabulary.append(word)
            return (word)
        else:
            return word

punctuation = string.punctuation
def preprocess(tweet):
    tweet = tweet.lower()
    tweet = re.sub('((www.[^s]+)|(https?://[^s]+))',' ',tweet)
    tweet = re.sub(r"@\S+", "", tweet)                
    tweet = nltk.word_tokenize(tweet)
    remove_punctuation = str.maketrans('', '', punctuation)
    tweet = [word.translate(remove_punctuation) for word in tweet if word not in basic_stopwords and word.isalnum() and not word.isdigit()]
#     tweet = [PorterStemmer().stem(word) for word in tweet]
#     tagged_tweet = nltk.pos_tag(tweet)
#     tweet = [custom_lemmatizer(word[0],word[1]) for word in tagged_tweet]
#     tweet = [word for word in tweet if word is not None]
    
    
    return tweet

start = timer()
df.tweet =df.tweet.apply(lambda x:preprocess(x)) 
end = timer()
print(end-start)

[nltk_data] Downloading package wordnet to /Users/antonis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/antonis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/antonis/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


267.219011526


In [3]:
#RUN FOR CUSTOM EMBEDDINGS
model = Word2Vec(df.tweet,window=4,workers=4,min_count=1)
all_normed_vectors = model.wv.get_normed_vectors()
model.save('model.bin')

filename = 'model.txt'
model.wv.save_word2vec_format(filename,binary=False)

word_vectors = {}

file = open('model.txt',encoding='utf-8')

for line in file:
    vector_values = line.split()
    vector = np.asarray(vector_values[1:])
    word_vectors[vector_values[0]] = vector

In [4]:
df['target'] = df['target'].replace(4,1)

tokenizer = Tokenizer()

tokenizer.fit_on_texts(df.tweet)

vocabulary = tokenizer.word_index #gia to word index

train_data, remaining_data = train_test_split(df, train_size=0.8,random_state=4324234)

X_rem = pad_sequences(tokenizer.texts_to_sequences(remaining_data.tweet),maxlen = 30)
y_rem = remaining_data.target

x_valid,x_test,y_valid,y_test = train_test_split(X_rem,y_rem, test_size=0.5,random_state=4354312)

x_train = pad_sequences(tokenizer.texts_to_sequences(train_data.tweet),maxlen = 30)
# x_test = pad_sequences(tokenizer.texts_to_sequences(test_data.tweet),maxlen = 30)


y_train = train_data.target
# y_test = test_data.target

In [5]:
#RUN FOR CUSTOM
num_words = len(vocabulary) + 1

embedded_matrix = np.zeros((num_words,100))
for word,i in vocabulary.items():
    embedded_vector = word_vectors.get(word)
    embedded_matrix[i] = embedded_vector


In [27]:
from keras.layers import LSTM, Conv1D, Dense, Flatten, MaxPooling1D, Dropout
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint

embedding_layer = tf.keras.layers.Embedding(num_words,100,weights=[embedded_matrix],
                                       input_length=30,trainable=False)

model = Sequential()
model.add(embedding_layer)
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 30, 100)           27144200  
                                                                 
 flatten_6 (Flatten)         (None, 3000)              0         
                                                                 
 dense_29 (Dense)            (None, 10)                30010     
                                                                 
 dense_30 (Dense)            (None, 1)                 11        
                                                                 
Total params: 27,174,221
Trainable params: 30,021
Non-trainable params: 27,144,200
_________________________________________________________________


In [28]:
hist = model.fit(x_train, y_train, epochs=40, batch_size=10, validation_split=0.2)


Epoch 1/40
  8230/102400 [=>............................] - ETA: 1:09 - loss: 0.5648 - accuracy: 0.7023

KeyboardInterrupt: 